In [1]:
% load_ext autoreload
% autoreload 2


In [2]:
import glob
import os
import pandas as pd
from collections import defaultdict
import datetime
import src.mex_helper as mex
import folium
from src.utils.map_vis import time_slider_choropleth, geojson_per_row

# average over weekday and weekend

In [28]:
%%time
aver = mex.stat_tw_dow_aver_hr_uniq_user('out')

stats dir: stats/MexTwHrUniqCnt-out/
loading stats by weekday or weekend
loading 0th file stats/MexTwHrUniqCnt-out/2009-10-01-located.csv
loading 50th file stats/MexTwHrUniqCnt-out/2009-11-20-located.csv
loading 100th file stats/MexTwHrUniqCnt-out/2010-01-09-located.csv
loading 150th file stats/MexTwHrUniqCnt-out/2010-03-27-located.csv
loading 200th file stats/MexTwHrUniqCnt-out/2010-05-16-located.csv
computing average of hourly vector by weekday or weekend
CPU times: user 4min 22s, sys: 1.06 s, total: 4min 23s
Wall time: 4min 25s


In [29]:
tw_avg_wd = pd.DataFrame(aver['wd']).T
tw_avg_wk = pd.DataFrame(aver['wk']).T

# distribute tower stats to grids

In [33]:
mex_t2g = mex.tower2grid('cities', 1000)

reading existing t2g file: data/mex_tower/mex_t2g_cities_1000m.csv


In [34]:
print('number of towers in cities has no call at all during weekday and weekend')
len(set(mex_t2g.gtid) - set(tw_avg_wd.index)), len(set(mex_t2g.gtid) - set(tw_avg_wk.index)),

number of towers in cities has no call at all during weekday and weekend


(233, 234)

In [35]:
def grid_avgerage(tw_avg, t2g):
    # there are grids without any call throughout the observation period
    g_avg = t2g.merge(tw_avg, left_on='gtid', right_index=True, how='left')

    for h in range(24):
        h=str(h)
        g_avg[h] = g_avg[h] * g_avg['weight']

    g_avg= g_avg.drop(['gtid','weight'], axis=1).groupby('grid').sum()  # fillna=0 by default

    return g_avg

In [36]:
g_avg_wd = grid_avgerage(tw_avg_wd, mex_t2g)
g_avg_wk = grid_avgerage(tw_avg_wk, mex_t2g)

# visualize by hour each city

In [37]:
mex_grids = mex.grids('cities', 1000)

reading existing grids


In [38]:
fake_dates = [f"1971-01-{i:02}" for i in range(1,25)]

In [39]:
mex_cities = mex.cities()

In [40]:
%%time
for dow, g_avg in [('wd', g_avg_wd), ('wk',g_avg_wk)]:
    print('day of week:', dow)
    for c in mex_cities.index:
        lon,lat = mex_cities.loc[c].geometry.centroid.coords[0]
        gc = mex_grids[mex_grids.city==c]
        gc_avg = g_avg.loc[gc.grid]
        values = gc_avg.T.to_dict('list')
        some_map = folium.Map(location=[lat,lon], zoom_start=10)
        time_slider_choropleth(gc, values, fake_dates, color_per_day=True).add_to(some_map)
        some_map.save(f'maps/MexGridChoropleth/{c}_{dow}.html')

day of week: wd
day of week: wk
CPU times: user 4min 38s, sys: 2.33 s, total: 4min 40s
Wall time: 5min


## keep hotspot only

In [41]:
from src.utils import loubar_thres

In [42]:
def keep_hotspot(gc_avg):
    for h in gc_avg:
        arr = gc_avg[h]
        loubar, arr_thres = loubar_thres(arr,is_sorted=False)
        gc_avg[h][gc_avg[h]<=arr_thres]=0
#         print(h, loubar, arr_thres)

In [43]:
%%time
for dow, g_avg in [('wd', g_avg_wd), ('wk',g_avg_wk)]:
    print('day of week:', dow)
    for c in mex_cities.index:
        lon,lat = mex_cities.loc[c].geometry.centroid.coords[0]
        gc = mex_grids[mex_grids.city==c]
        gc_avg = g_avg.loc[gc.grid].copy()
        keep_hotspot(gc_avg)
        values = gc_avg.T.to_dict('list')
        some_map = folium.Map(location=[lat,lon], zoom_start=10)
        time_slider_choropleth(gc, values, fake_dates, color_per_day=True).add_to(some_map)
#         break
        some_map.save(f'maps/MexGridChoroplethHotSpot/{c}_{dow}.html')
#     break

day of week: wd
day of week: wk
CPU times: user 4min 23s, sys: 1.2 s, total: 4min 24s
Wall time: 4min 26s


## hotspot persistency

In [46]:
from branca.colormap import linear
cmap = linear.Reds_09.scale(0,24)

In [49]:
# %%time
for dow, g_avg in [('wd', g_avg_wd), ('wk',g_avg_wk)]:
    print('day of week:', dow)
    
    some_map = folium.Map(location=[mex.CLAT,mex.CLON], zoom_start=8)
    
    for c in mex_cities.index:
        cgeom=mex_cities.loc[c].geometry
        lon,lat = cgeom.centroid.coords[0]
        gc = mex_grids[mex_grids.city==c].copy()
        gc_avg = g_avg.loc[gc.grid].copy()
        keep_hotspot(gc_avg)
        gc['persistency'] = (gc_avg!=0).sum(axis=1)
        gc = gc[gc.persistency!=0]
        gc = gc.append({'grid':-1, 'geometry':cgeom, 'persistency':0},ignore_index=True).sort_values('grid')
        gjson = gc[['grid','persistency','geometry']].to_json()
        folium.GeoJson(
            gjson,
            name=c,
            style_function=lambda feature: {
                'fillColor': cmap(feature['properties']['persistency']),
                'color': 'black',
                'weight': 1,
                'dashArray': '5, 5',
                'fillOpacity': 0.8,
            }
        ).add_to(some_map)
#         break
    
    cmap.caption = 'hotspot persistency (hours)'
    cmap.add_to(some_map)
    folium.LayerControl(collapsed=False).add_to(some_map)
    some_map.save(f'maps/MexGridChoroplethHotSpotPersistency_{dow}.html')
#     break


day of week: wd
day of week: wk
